# UNIPD - Deep Learning 2025 - Challenge 2: Weather Forecasting

In this challenge, you will develop a deep learning model to forecast weather data based on historical observations collected from various stations.

You are provided with a dataset consisting of daily weather measurements over time, for multiple stations. Your task is to forecast future weather values for each station over a 30-day horizon.


## Loading data

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [2]:
data_dir = '/kaggle/input/unipd-deep-learning-2025-challenge-2/'
data = pd.read_csv(data_dir + 'train_dataset.csv', index_col=[0, 1])
stations = [station.values for _, station in data.groupby(level=0)]
data_arr = np.stack(stations, axis=0)

In [3]:
n_stations, n_days, n_features = data_arr.shape
print(f"Number of weather stations: {n_stations}")
print(f"Number of days of data: {n_days}")
print(f"Number of weather variables: {n_features}")
print(data_arr.shape)  

Number of weather stations: 422
Number of days of data: 695
Number of weather variables: 76
(422, 695, 76)


# Dataset Definition and Split

In [4]:
# Data Augmentation Functions
from scipy.interpolate import interp1d

def time_warp(x, scale_range=(0.8, 1.2)):
    original_len = x.shape[0]
    scale = np.random.uniform(*scale_range)
    new_len = int(original_len * scale)

    time_steps = np.linspace(0, original_len - 1, num=new_len)
    interpolated = interp1d(np.arange(original_len), x, axis=0, kind='linear', fill_value='extrapolate')
    x_warped = interpolated(time_steps)

    # Back to original length via interpolation or truncation/padding
    final_interp = interp1d(np.linspace(0, 1, num=new_len), x_warped, axis=0, fill_value='extrapolate')
    x_final = final_interp(np.linspace(0, 1, num=original_len))
    return x_final

def permute_segments(x, n_segments=3):
    segs = np.array_split(x, n_segments, axis=0)
    np.random.shuffle(segs)
    return np.concatenate(segs, axis=0)

def magnitude_warp(x, sigma=0.2, knot=4):
    from scipy.interpolate import CubicSpline
    time = np.arange(x.shape[0])
    warp = np.random.normal(loc=1.0, scale=sigma, size=(knot, x.shape[1]))
    cs = CubicSpline(np.linspace(0, x.shape[0]-1, num=knot), warp, axis=0)
    warping_curve = cs(time)
    return x * warping_curve

In [5]:
class WeatherDataset(Dataset):
    def __init__(self, data, seq_len, forecast_len, normalize=True, augment=True):
        self.samples = []
        self.seq_len = seq_len
        self.forecast_len = forecast_len
        self.augment = augment

        for station in data:
            # Normalization by station
            if normalize:
                mu    = station.mean(axis=0)        # (n_features,)
                sigma = station.std(axis=0) + 1e-8  # (n_features,)
            else:
                mu, sigma = np.zeros(station.shape[1]), np.ones(station.shape[1])

            n = len(station)
            max_i = n - seq_len - forecast_len + 1

            for i in range(max_i):
                x = station[i : i + seq_len].copy()
                y = station[i + seq_len : i + seq_len + forecast_len].copy()
                if normalize:
                    x = (x - mu) / sigma
                    y = (y - mu) / sigma

                self.samples.append((x, y, mu, sigma))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y, mu, sigma = self.samples[idx]

        if self.augment:
            # Gaussian noise
            x += np.random.normal(0, 0.01, x.shape)

            # Random scaling
            x *= np.random.uniform(0.95, 1.05)

            # Time warping
            if np.random.rand() < 0.3:
                x = time_warp(x)

            # Permutation
            if np.random.rand() < 0.3:
                x = permute_segments(x)

            # Magnitude warping
            if np.random.rand() < 0.3:
                x = magnitude_warp(x)

        return (
            torch.tensor(x, dtype=torch.float32),
            torch.tensor(y, dtype=torch.float32),
            torch.tensor(mu, dtype=torch.float32),
            torch.tensor(sigma, dtype=torch.float32),
        )


In [6]:
def split_dataset(data_arr, val_ratio=0.2):
    n_stations, n_days, n_features = data_arr.shape

    # 20% of time for validation
    split_idx = int(n_days * (1 - val_ratio))

    train_data = data_arr[:, :split_idx, :]
    val_data = data_arr[:, split_idx:, :]

    print(f"Train data shape: {train_data.shape}")
    print(f"Validation data shape: {val_data.shape}")

    return train_data, val_data

# Model Definition

In [7]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.1):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)
        
    def forward(self, x):
        # x: [batch_size, seq_len, input_size]
        outputs, (h, c) = self.lstm(x)
        return h, c  # return hidden and cell state
        

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.1):
        super(Decoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h, c):
        # x: [batch_size, 1, input_size]
        output, (h, c) = self.lstm(x, (h, c))
        prediction = self.fc(output)  # [batch_size, 1, output_size]
        return prediction, h, c


# Seq2Seq combines encoder and decoder for full forecasting
class WeatherModel(nn.Module):
    def __init__(self, input_size, hidden_size, forecast_horizon, num_layers=1, dropout=0.1):
        super(WeatherModel, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, num_layers, dropout)
        self.decoder = Decoder(input_size, hidden_size, input_size, num_layers, dropout)
        self.forecast_horizon = forecast_horizon
        self.input_size = input_size

    def forward(self, src):
        # src: [batch_size, seq_len, input_size]
        batch_size = src.size(0)

        # Encode the input sequence
        h, c = self.encoder(src)

        # First decoder input is the last input time step
        decoder_input = src[:, -1:, :]  # [batch_size, 1, input_size]
        outputs = []

        for _ in range(self.forecast_horizon):
            out, h, c = self.decoder(decoder_input, h, c)  # out: [batch, 1, input_size]
            outputs.append(out)
            decoder_input = out  # feeding back prediction

        outputs = torch.cat(outputs, dim=1)  # [batch_size, forecast_horizon, input_size]
        return outputs

# Training

In [8]:
def calculate_scale(data):
    diffs = np.abs(data[:, 1:, :] - data[:, :-1, :])
    return np.mean(diffs)

In [9]:
def train_model(model, train_loader, val_loader, device, naive_scale, num_epochs=50):
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5)

    best_mase = float('inf')
    patience_counter = 0
    patience = 5#10

    for epoch in range(1, num_epochs + 1):
        # ----- Training -----
        model.train()
        total_loss = 0.0
        count = 0

        for x_b, y_b, _, _ in train_loader:
            x_b, y_b = x_b.to(device), y_b.to(device)
            optimizer.zero_grad()
            preds = model(x_b)
            loss = criterion(preds, y_b)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()
            count += 1

        avg_train_loss = total_loss / count

        # ----- Validation -----
        model.eval()
        val_loss = 0.0
        vcount = 0
        all_preds, all_targets, all_mu, all_sig = [], [], [], []

        with torch.no_grad():
            for x_b, y_b, mu_b, sig_b in val_loader:
                x_b, y_b = x_b.to(device), y_b.to(device)
                mu_b, sig_b = mu_b.to(device), sig_b.to(device)

                preds = model(x_b)
                loss = criterion(preds, y_b)

                val_loss += loss.item()
                vcount += 1

                all_preds.append(preds.cpu())
                all_targets.append(y_b.cpu())
                all_mu.append(mu_b.cpu())
                all_sig.append(sig_b.cpu())

        avg_val_loss = val_loss / vcount

        # ----- Compute MASE -----
        preds = torch.cat(all_preds, dim=0)
        targets = torch.cat(all_targets, dim=0)
        mus = torch.cat(all_mu, dim=0).unsqueeze(1)
        sigs = torch.cat(all_sig, dim=0).unsqueeze(1)

        preds_denorm = preds * sigs + mus
        targets_denorm = targets * sigs + mus

        mae = torch.abs(preds_denorm - targets_denorm).mean().item()
        mase = mae / naive_scale

        scheduler.step(mase)
        lr = optimizer.param_groups[0]['lr']

        print(f"Epoch {epoch}/{num_epochs} | TrainLoss: {avg_train_loss:.4f} | "
              f"ValLoss: {avg_val_loss:.4f} | ValMASE: {mase:.4f} | LR: {lr:.6f}")

        # ----- Early Stopping -----
        if mase < best_mase:
            best_mase = mase
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pt')
            print("Best model saved")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch}. Best MASE: {best_mase:.4f}")
                break

    return best_mase


# Main Function

In [10]:
# General Parameters
input_size = data_arr.shape[2]
output_size = input_size
batch_size = 64
seq_length = 80   # Input sequence length
forecast_horizon = 30  # Number of time steps to predict

#-------- Dataset Initialization ---------------#
# Splitting Dataset
train_arr, val_arr = split_dataset(data_arr, val_ratio=0.2)

# Calculate naive scale before creating datasets
naive_scale = calculate_scale(train_arr)
print(f"Forecast scale: {naive_scale:.6f}")

# Create datasets with normalization
train_dataset = WeatherDataset(train_arr, seq_length, forecast_horizon, normalize=True, augment=True)
val_dataset = WeatherDataset(val_arr, seq_length, forecast_horizon, normalize=True, augment=False)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                        num_workers=0, pin_memory=True if device.type=='cuda' else False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                      num_workers=0, pin_memory=True if device.type=='cuda' else False)


#----------Model and Training------------------#
# Model
model = WeatherModel(
    input_size=input_size,
    hidden_size=64,
    num_layers=2,
    forecast_horizon=forecast_horizon,
    dropout=0.4
).to(device)

# Train
best_mase = train_model(model, train_loader, val_loader, device, naive_scale, 80)

Train data shape: (422, 556, 76)
Validation data shape: (422, 139, 76)
Forecast scale: 4.907250
Epoch 1/80 | TrainLoss: 0.5340 | ValLoss: 0.4917 | ValMASE: 1.1985 | LR: 0.000010
Best model saved
Epoch 2/80 | TrainLoss: 0.4990 | ValLoss: 0.4892 | ValMASE: 1.1948 | LR: 0.000010
Best model saved
Epoch 3/80 | TrainLoss: 0.4977 | ValLoss: 0.4890 | ValMASE: 1.1945 | LR: 0.000010
Best model saved
Epoch 4/80 | TrainLoss: 0.4971 | ValLoss: 0.4888 | ValMASE: 1.1939 | LR: 0.000010
Best model saved
Epoch 5/80 | TrainLoss: 0.4961 | ValLoss: 0.4897 | ValMASE: 1.1970 | LR: 0.000010
Epoch 6/80 | TrainLoss: 0.4945 | ValLoss: 0.4900 | ValMASE: 1.2002 | LR: 0.000010
Epoch 7/80 | TrainLoss: 0.4930 | ValLoss: 0.4907 | ValMASE: 1.2052 | LR: 0.000010
Epoch 8/80 | TrainLoss: 0.4913 | ValLoss: 0.4910 | ValMASE: 1.2091 | LR: 0.000010
Epoch 9/80 | TrainLoss: 0.4896 | ValLoss: 0.4921 | ValMASE: 1.2144 | LR: 0.000010
Early stopping at epoch 9. Best MASE: 1.1939


## Predictions

In [11]:
# Load the best model
model.load_state_dict(torch.load("best_model.pt"))
model.eval()
model.to(device)

# Prepare last input sequence for each station
x_input = data_arr[:, -seq_length:, :]

# Compute per-station normalization stats
mus = data_arr.mean(axis=1, keepdims=True)
sigmas = data_arr.std(axis=1, keepdims=True) + 1e-8

# Normalize input sequences
x_input_norm = (x_input - mus) / sigmas
x_input_tensor = torch.tensor(x_input_norm, dtype=torch.float32).to(device)

# Predictions with normalized data
with torch.no_grad():
    y_pred_norm = model(x_input_tensor).cpu().numpy()  # (n_stations, forecast_len, n_features)

# Denormalize predictions
y_pred = y_pred_norm * sigmas + mus

# Submissions
n_stations, n_forecast_steps, n_features = y_pred.shape
submission_data = []

for station_id in range(n_stations):
    for time_step in range(n_forecast_steps):
        submission_id = f'{station_id}_{time_step}'
        row = y_pred[station_id, time_step, :]
        submission_data.append({
            'id': submission_id,
            **{f'var{i+1}': row[i] for i in range(n_features)}  # var1, var2, ...
        })

submission_df = pd.DataFrame(submission_data).set_index('id')

# 8. Save to CSV
submission_df.to_csv("submission.csv")

print("Submission file saved as 'submission.csv'")


Submission file saved as 'submission.csv'
